## Code for performing evaluation with ViFi-CLIP on custom datasets

### Please set the corresponding values in the cell below. Afterwards, just run the cells for inference with ViFi-CLIP model

In [1]:
import pathlib
# Class label csv path
labels_csv_path = '../datasets/home_labels.csv'
# Datset folder
dataset_root_path = pathlib.Path('../datasets')
# output result csv path
output_csv_path = '../outputs/vclip_results.csv'

In [2]:
### Set values here ###
config = 'configs/zero_shot/train/k400/16_16_vifi_clip.yaml'
output_folder_name = "outputs"
pretrained_model_path = "ckpts/vifi_clip_10_epochs_k400_full_finetuned.pth"

### Import libraries 

In [3]:
import torch
import torch.nn as nn
from utils.config import get_config
from utils.logger import create_logger
import time
import numpy as np
from utils.config import get_config
from trainers import vificlip
from datasets.pipeline import *

### Setting up configuration, no need to change anything.

In [4]:
# Step 1:
# Configuration class 
class parse_option():
    def __init__(self):
        self.config = config
        self.output =  output_folder_name   # Name of output folder to store logs and save weights
        self.resume = pretrained_model_path
        # No need to change below args.
        self.only_test = True
        self.opts = None
        self.batch_size = None
        self.pretrained = None
        self.accumulation_steps = None
        self.local_rank = 0
args = parse_option()
config = get_config(args)
# logger
logger = create_logger(output_dir=args.output, name=f"{config.MODEL.ARCH}")
logger.info(f"working dir: {config.OUTPUT}")

=> merge config from configs/zero_shot/train/k400/16_16_vifi_clip.yaml
[2024-04-16 15:52:18 ViT-B/16](588939814.py 19): INFO working dir: outputs


### Read Class Labels

In [5]:
import csv

label2id = {}
id2label = {}
with open(labels_csv_path, mode='r') as file:
    csv_reader = csv.reader(file)

    for i, row in enumerate(csv_reader):
        class_name = row[0]
        label2id[class_name] = i
        id2label[i] = class_name

class_labels = list(label2id.keys())

print(f"{len(class_labels)} Unique classes: {class_labels}.")

70 Unique classes: ['No people is in the room', 'Studying', 'Typing', 'Using computer', 'Making phone calls', 'Play with phone/tablet', 'Playing with pets', 'Sitting down', 'Reading', 'Writing', 'Setting table', 'Eating food', 'Dining', 'Cooking', 'Sleeping', 'Laying down', 'Picking up objects', 'Brushing teeth', 'Showering', 'Playing music', 'TV', 'Cleaning', 'Doing laundry', 'Mopping floor', 'Vacumning', 'Organizing space', 'Sewing', 'Knitting', 'Decorating', 'Party', 'Chatting', 'Talking', 'Singing', 'Laughing', 'Speaking', 'Dancing', 'Drinking', 'Meditating', 'Drawing', 'Painting', 'Playing board games', 'Playing video games', 'Taking photos', 'Potluck', 'Working', 'Exercising', 'Walking', 'Running', 'Celebrating', 'Physical altercations', 'Verbal confrontations', 'Using drug', 'Theft or vandalism', 'Fighting', 'Domestic violence', 'Break in', 'Glass breaking', 'Unattended cooking', 'Smoke or Fire or Open flame', 'Smoking', 'Gunshot', 'Making noise', 'Falling down', 'Tripping', 'Cr

### Loading ViFi-CLIP and its pretrained weights

In [6]:
# Step 2:
# Create the ViFi-CLIP models and load pretrained weights
model = vificlip.returnCLIP(config,
                            logger=logger,
                            class_names=class_labels,)
model = model.float().cuda()  # changing to cuda here


[2024-04-16 15:52:18 ViT-B/16](vificlip.py 204): INFO Loading CLIP (backbone: ViT-B/16)
[2024-04-16 15:52:20 ViT-B/16](vificlip.py 207): INFO Building ViFi-CLIP CLIP
[2024-04-16 15:52:20 ViT-B/16](vificlip.py 224): INFO Turning on gradients for COMPLETE ViFi-CLIP model
[2024-04-16 15:52:20 ViT-B/16](vificlip.py 247): INFO Parameters to be updated: {'image_encoder.transformer.resblocks.5.attn.in_proj_weight', 'image_encoder.transformer.resblocks.2.mlp.c_proj.weight', 'image_encoder.transformer.resblocks.5.attn.out_proj.bias', 'text_encoder.transformer.resblocks.5.mlp.c_proj.weight', 'image_encoder.transformer.resblocks.11.attn.out_proj.bias', 'image_encoder.transformer.resblocks.2.attn.in_proj_weight', 'image_encoder.transformer.resblocks.3.attn.in_proj_bias', 'image_encoder.transformer.resblocks.9.attn.out_proj.bias', 'text_encoder.transformer.resblocks.3.mlp.c_proj.bias', 'text_encoder.transformer.resblocks.6.mlp.c_proj.bias', 'text_encoder.transformer.resblocks.7.attn.in_proj_weight'

In [7]:
logger.info(f"==============> Resuming form {config.MODEL.RESUME}....................")
checkpoint = torch.load(config.MODEL.RESUME, map_location='cpu')
load_state_dict = checkpoint['model']
# now remove the unwanted keys:
if "module.prompt_learner.token_prefix" in load_state_dict:
    del load_state_dict["module.prompt_learner.token_prefix"]

if "module.prompt_learner.token_suffix" in load_state_dict:
    del load_state_dict["module.prompt_learner.token_suffix"]

if "module.prompt_learner.complete_text_embeddings" in load_state_dict:
    del load_state_dict["module.prompt_learner.complete_text_embeddings"]
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in load_state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

[2024-04-16 15:52:20 ViT-B/16](2665388570.py 1): INFO ==============> Resuming form ckpts/vifi_clip_10_epochs_k400_full_finetuned.pth....................


In [8]:
# load params
msg = model.load_state_dict(new_state_dict, strict=False)
logger.info(f"resume model: {msg}")

[2024-04-16 15:52:20 ViT-B/16](3852643250.py 3): INFO resume model: _IncompatibleKeys(missing_keys=['prompt_learner.complete_text_embeddings'], unexpected_keys=[])


### Preprocessing input video 

In [9]:
# Step 3: 
# Preprocessing for video
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_bgr=False)
scale_resize = int(256 / 224 * config.DATA.INPUT_SIZE)
val_pipeline = [
    dict(type='DecordInit'),
    dict(type='SampleFrames', clip_len=1, frame_interval=1, num_clips=config.DATA.NUM_FRAMES, test_mode=True),
    dict(type='DecordDecode'),
    dict(type='Resize', scale=(-1, scale_resize)),
    dict(type='CenterCrop', crop_size=config.DATA.INPUT_SIZE),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='FormatShape', input_format='NCHW'),
    dict(type='Collect', keys=['imgs'], meta_keys=[]),
    dict(type='ToTensor', keys=['imgs'])
]
if config.TEST.NUM_CROP == 3:
    val_pipeline[3] = dict(type='Resize', scale=(-1, config.DATA.INPUT_SIZE))
    val_pipeline[4] = dict(type='ThreeCrop', crop_size=config.DATA.INPUT_SIZE)
if config.TEST.NUM_CLIP > 1:
    val_pipeline[1] = dict(type='SampleFrames', clip_len=1, frame_interval=1, num_clips=config.DATA.NUM_FRAMES, multiview=config.TEST.NUM_CLIP)
pipeline = Compose(val_pipeline)

### Construct dataset

In [10]:
train_paths = list(dataset_root_path.glob("train/*/*"))
# test_paths = list(dataset_root_path.glob("test/*/*.mp4"))


def construct_dataset(file_paths):
    dataset = []
    for path in file_paths:
        label = str(path).split("\\")[-2]
        dataset.append({'filename': path, 'tar': False, 'modality': 'RGB', 'start_index': 0, 'label': label2id[label]})
    return dataset

train_dataset = construct_dataset(train_paths)
print('train_dataset', len(train_dataset), train_dataset[0])
# test_dataset = construct_dataset(test_paths)
# print('test_dataset', len(test_dataset), test_dataset[0])

train_dataset 424 {'filename': WindowsPath('../datasets/train/Crying/9q8mf6GUiiE_000009_000019.mp4'), 'tar': False, 'modality': 'RGB', 'start_index': 0, 'label': 64}


### ViFi-CLIP inference with given video

In [11]:
from tqdm import tqdm
from collections import defaultdict
import csv
import torch.nn.functional as F

def inference(dataset):
    correct = defaultdict(int)
    total = defaultdict(int)

    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["file path", "ground truth", "top1 pred", "confidence", "top2 pred", "confidence", "top3 pred", "confidence", "top4 pred", "confidence", "top5 pred", "confidence"])
        for i, dict_file in tqdm(enumerate(dataset), total=len(dataset)):
            label = int(dict_file['label'])
            out_row = [dict_file['filename'], label]
            total[label] += 1

            if not str(dict_file['filename']).endswith(".mp4"):
                out_row.extend([-1, 0] * 5) # use -1 if no audio
                writer.writerow(out_row)
                continue

            # Inference through ViFi-CLIP
            video = pipeline(dict_file)
            video_tensor = video['imgs'].unsqueeze(0).cuda().float()
            
            with torch.no_grad():
                with torch.cuda.amp.autocast():
                    similarities, _, _, _ = model(video_tensor)
            similarity = F.softmax(similarities, dim=1)
            pred_index = similarity.argmax(-1).item()

            values, indices = similarity[0].topk(5)
            pred_index = indices[0]

            # Output the results
            
            # print("\nFile:", dict_file['filename'])
            # print(f"Ground Truth: {class_labels[label]}")
            # print("Top predictions:")
            for value, index in zip(values, indices):
                # print(f"{class_labels[index]:>16s}: {100 * value.item():.2f}%")
                out_row.append(index.item())
                out_row.append(round(value.item() * 100, 4))
            print("")
            writer.writerow(out_row)

            if label == pred_index: 
                correct[label] += 1
            

        

    return correct, total
    

In [12]:
correct, total = inference(train_dataset)

for label in total:
    print(f"{class_labels[label]}: {correct[label]} / {total[label]}")

  0%|          | 1/424 [00:07<49:49,  7.07s/it]

  1%|          | 3/424 [00:07<13:04,  1.86s/it]

  1%|          | 4/424 [00:07<08:12,  1.17s/it]

  1%|▏         | 6/424 [00:08<04:55,  1.41it/s]

  2%|▏         | 8/424 [00:09<02:56,  2.35it/s]

...

 99%|█████████▉| 421/424 [01:48<00:01,  1.51it/s]

100%|█████████▉| 422/424 [01:48<00:01,  1.59it/s]

100%|█████████▉| 423/424 [01:49<00:00,  1.84it/s]

100%|██████████| 424/424 [01:49<00:00,  3.87it/s]


Crying: 48 / 50
Falling down: 7 / 30
Glass breaking: 1 / 53
Gunshot: 0 / 51
Normal: 0 / 200
Smoke or Fire or Open flame: 34 / 40


In [14]:
for label in total:
    print(f"{class_labels[label]}: {correct[label]} / {total[label]}")

Crying: 48 / 50
Falling down: 7 / 30
Glass breaking: 1 / 53
Gunshot: 0 / 51
Normal: 0 / 200
Smoke or Fire or Open flame: 34 / 40
